# Data Crawling

In [ ]:
# Real-time order book data from Binance platform
import ccxt
import time
import csv

if __name__ == "__main__":
    exchange = ccxt.binance()
    symbol = "ETH/USDT"
    with open("data.csv", "w", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(['data_type', 'price', 'volume', 'time'])
        while True:
            current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            print(current_time)
            try:
                data = exchange.fetch_l2_order_book(symbol)
            except Exception as e:
                pass
            else:
                bids_data = data['bids']
                asks_data = data['asks']
                for item in bids_data:
                    writer.writerow(['bids', item[0], item[1], current_time])
                for item in asks_data:
                    writer.writerow(['asks', item[0], item[1], current_time])

            time.sleep(1)

# Variable Calculation (Binance Real-time data)

In [1]:
import pandas as pd
df = pd.read_csv("binance_orderbook_0.5h.csv")
df.head()

,data_type,price,volume,time
0,bids,2015.07,10.40200,2021-06-21 16:39:13
1,bids,2015.06,0.03883,2021-06-21 16:39:13
2,bids,2015.01,1.00000,2021-06-21 16:39:13
3,bids,2015.00,0.50000,2021-06-21 16:39:13
4,bids,2014.92,0.37362,2021-06-21 16:39:13


In [20]:
# Match ask and bid
df_ask = df[df['data_type']=='asks']
df_ask = df_ask.reset_index()
df_ask = df_ask.rename(columns = {'price':'price_ask'})
df_ask=df_ask.rename(columns = {'volume':'volume_ask'})
df_ask=df_ask.rename(columns = {'time':'time_ask'})

df_bid = df[df['data_type']=='bids']
df_bid = df_bid.reset_index()
df_bid=df_bid.rename(columns = {'data_type':'data_type1'})
df_bid=df_bid.rename(columns = {'price':'price_bid'})
df_bid=df_bid.rename(columns = {'volume':'volume_bid'})
df_bid=df_bid.rename(columns = {'time':'time_bid'})
df_bid=df_bid.rename(columns = {'index':'index_bid'})

df_orderbook = pd.concat([df_ask,df_bid],axis=1)
df_orderbook = df_orderbook.drop(['index','index_bid','time_ask'],axis=1)
df_orderbook=df_orderbook.rename(columns = {'time_bid':'time'})

155500

In [64]:
# Time variable pre-processing
import datetime
df_orderbook['time'] = pd.to_datetime(df_orderbook['time'])
df_orderbook['time_nodate']=df_orderbook['time'].dt.time
df_orderbook['time_tomin']=df_orderbook['time_nodate'].apply(lambda x: str(x)[:-3])

# Get the time to minute
df_orderbook['time_tomin'] = pd.to_datetime(df_orderbook['time_tomin']).dt.time
df_orderbook

,data_type,price_ask,volume_ask,data_type1,price_bid,volume_bid,time,time_nodate,time_tomin
0,asks,2015.08,10.44232,bids,2015.07,10.40200,2021-06-21 16:39:13,16:39:13,16:39:00
1,asks,2015.12,0.04625,bids,2015.06,0.03883,2021-06-21 16:39:13,16:39:13,16:39:00
2,asks,2015.20,0.40000,bids,2015.01,1.00000,2021-06-21 16:39:13,16:39:13,16:39:00
3,asks,2015.31,0.06225,bids,2015.00,0.50000,2021-06-21 16:39:13,16:39:13,16:39:00
4,asks,2015.32,1.50000,bids,2014.92,0.37362,2021-06-21 16:39:13,16:39:13,16:39:00
...,...,...,...,...,...,...,...,...,...
155495,asks,2010.26,0.03397,bids,2002.80,2.00000,2021-06-21 17:12:39,17:12:39,17:12:00
155496,asks,2010.34,0.24973,bids,2002.74,0.42203,2021-06-21 17:12:39,17:12:39,17:12:00
155497,asks,2010.36,9.04077,bids,2002.67,0.48214,2021-06-21 17:12:39,17:12:39,17:12:00
155498,asks,2010.37,0.02490,bids,2002.65,2.00000,2021-06-21 17:12:39,17:12:39,17:12:00


### Ask-bid Spread

In [75]:
# Average ABS calculation (delta t = 1 minute)
df_orderbook['ask-bid spread'] = df_orderbook['price_ask']-df_orderbook['price_bid']
df_ABS = pd.DataFrame(df_orderbook['ask-bid spread'].groupby(df_orderbook['time_tomin']).mean())
df_ABS=df_ABS.rename(columns = {'ask-bid spread':'ABS_ave'})

In [76]:
df_ABS

,ABS_ave
time_tomin,
16:39:00,3.523412
16:40:00,3.219809
16:41:00,3.638836
16:42:00,3.756940
16:43:00,3.654773
16:44:00,3.513657
16:45:00,3.661869
16:46:00,3.592769
16:47:00,3.801893
